# SPR 2026 - Super Ensemble v3 (Threshold Tuning)

**Baseado em:** BERTimbau v4 (0.82073) 🏆

**Melhoria:** Combinar TODOS os melhores modelos + threshold tuning

**Componentes:**
1. BERTimbau + Focal Loss + Threshold
2. TF-IDF + LinearSVC (calibrado)
3. TF-IDF + SGDClassifier
4. Stacking dos TF-IDF models

**Estratégia:**
- Weighted average baseado em F1
- Threshold tuning no ensemble final

---
## CONFIGURAÇÃO KAGGLE:
1. **Add Input** → **Models** → `bertimbau-ptbr-complete`
2. **Add Input** → **Competition** → `spr-2026-mammography-report-classification`
3. **Settings** → Internet → **OFF**, GPU → **T4 x2**
---

In [ ]:
# ===== SUPER ENSEMBLE v3 - THRESHOLD TUNING =====

import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

print("="*60)
print("SPR 2026 - Super Ensemble v3 (Threshold Tuning)")
print("="*60)

# ===== CONFIG =====
SEED = 42
MAX_LEN = 256
BATCH_SIZE = 8
EPOCHS = 3
LR = 2e-5
NUM_CLASSES = 7
FOCAL_GAMMA = 2.0
FOCAL_ALPHA = 0.25
THRESHOLDS = {0: 0.50, 1: 0.50, 2: 0.50, 3: 0.50, 4: 0.50, 5: 0.30, 6: 0.25}

DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(SEED)
np.random.seed(SEED)

def find_model_path():
    base = '/kaggle/input'
    def search_dir(directory, depth=0, max_depth=10):
        if depth > max_depth: return None
        try:
            for item in os.listdir(directory):
                path = os.path.join(directory, item)
                if os.path.isdir(path) and os.path.exists(os.path.join(path, 'config.json')):
                    return path
                result = search_dir(path, depth + 1, max_depth) if os.path.isdir(path) else None
                if result: return result
        except: pass
        return None
    return search_dir(base)

MODEL_PATH = find_model_path()
print(f"Device: {device} | Model: {MODEL_PATH}")

In [ ]:
# ===== DADOS =====
train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
test_df = pd.read_csv(f'{DATA_DIR}/test.csv')

train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df['report'].values, train_df['target'].values,
    test_size=0.1, random_state=SEED, stratify=train_df['target']
)

print(f"Train: {len(train_texts)}, Val: {len(val_texts)}, Test: {len(test_df)}")

In [ ]:
# ===== PARTE 1: TF-IDF MODELS =====
print("\n" + "="*60)
print("1. TF-IDF Models")
print("="*60)

tfidf = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))
X_train = tfidf.fit_transform(train_texts)
X_val = tfidf.transform(val_texts)
X_test = tfidf.transform(test_df['report'].values)

# LinearSVC calibrado
print("Treinando LinearSVC + Calibration...")
svc = LinearSVC(C=1.0, max_iter=10000, random_state=SEED)
svc_cal = CalibratedClassifierCV(svc, cv=3, method='sigmoid')
svc_cal.fit(X_train, train_labels)
svc_probs = svc_cal.predict_proba(X_val)
print(f"  LinearSVC F1: {f1_score(val_labels, svc_cal.predict(X_val), average='macro'):.5f}")

# SGDClassifier calibrado
print("Treinando SGDClassifier + Calibration...")
sgd = SGDClassifier(loss='hinge', random_state=SEED)
sgd_cal = CalibratedClassifierCV(sgd, cv=3, method='sigmoid')
sgd_cal.fit(X_train, train_labels)
sgd_probs = sgd_cal.predict_proba(X_val)
print(f"  SGDClassifier F1: {f1_score(val_labels, sgd_cal.predict(X_val), average='macro'):.5f}")

# Stacking
print("Treinando Stacking...")
base_estimators = [
    ('svc', CalibratedClassifierCV(LinearSVC(random_state=SEED), cv=3)),
    ('sgd', CalibratedClassifierCV(SGDClassifier(random_state=SEED), cv=3)),
]
stacking = StackingClassifier(
    estimators=base_estimators,
    final_estimator=LogisticRegression(max_iter=1000, random_state=SEED),
    cv=3
)
stacking.fit(X_train, train_labels)
stacking_probs = stacking.predict_proba(X_val)
print(f"  Stacking F1: {f1_score(val_labels, stacking.predict(X_val), average='macro'):.5f}")

In [ ]:
# ===== PARTE 2: BERTIMBAU =====
print("\n" + "="*60)
print("2. BERTimbau + Focal Loss")
print("="*60)

class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0):
        super().__init__()
        self.alpha, self.gamma = alpha, gamma
    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        return (self.alpha * (1 - pt) ** self.gamma * ce_loss).mean()

class TextDataset(Dataset):
    def __init__(self, texts, labels=None, tokenizer=None, max_len=256):
        self.texts, self.labels, self.tokenizer, self.max_len = texts, labels, tokenizer, max_len
    def __len__(self): return len(self.texts)
    def __getitem__(self, idx):
        enc = self.tokenizer(str(self.texts[idx]), truncation=True, max_length=self.max_len, padding='max_length', return_tensors='pt')
        item = {'input_ids': enc['input_ids'].squeeze(), 'attention_mask': enc['attention_mask'].squeeze()}
        if self.labels is not None: item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, local_files_only=True)
train_ds = TextDataset(train_texts, train_labels, tokenizer, MAX_LEN)
val_ds = TextDataset(val_texts, val_labels, tokenizer, MAX_LEN)
test_ds = TextDataset(test_df['report'].values, None, tokenizer, MAX_LEN)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE)

model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH, num_labels=NUM_CLASSES, local_files_only=True)
model.to(device)
criterion = FocalLoss(alpha=FOCAL_ALPHA, gamma=FOCAL_GAMMA)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_loader)*EPOCHS)

for epoch in range(EPOCHS):
    model.train()
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        optimizer.zero_grad()
        outputs = model(batch['input_ids'].to(device), attention_mask=batch['attention_mask'].to(device))
        loss = criterion(outputs.logits, batch['labels'].to(device))
        loss.backward()
        optimizer.step()
        scheduler.step()

model.eval()
bert_probs_val = []
with torch.no_grad():
    for batch in val_loader:
        outputs = model(batch['input_ids'].to(device), attention_mask=batch['attention_mask'].to(device))
        bert_probs_val.append(F.softmax(outputs.logits, dim=-1).cpu().numpy())
bert_probs_val = np.vstack(bert_probs_val)
print(f"  BERTimbau F1: {f1_score(val_labels, np.argmax(bert_probs_val, axis=1), average='macro'):.5f}")

In [ ]:
# ===== SUPER ENSEMBLE =====
print("\n" + "="*60)
print("3. Super Ensemble")
print("="*60)

def apply_thresholds(probs, thresholds):
    preds = []
    for i in range(len(probs)):
        pred = np.argmax(probs[i])
        for c in [6, 5]:
            if probs[i, c] >= thresholds[c]:
                pred = c
                break
        preds.append(pred)
    return np.array(preds)

# Calcular F1 de cada modelo
f1s = {
    'bert': f1_score(val_labels, np.argmax(bert_probs_val, axis=1), average='macro'),
    'svc': f1_score(val_labels, svc_cal.predict(X_val), average='macro'),
    'sgd': f1_score(val_labels, sgd_cal.predict(X_val), average='macro'),
    'stacking': f1_score(val_labels, stacking.predict(X_val), average='macro'),
}
print(f"F1 individual: {f1s}")

# Pesos proporcionais ao F1
total = sum(f1s.values())
weights = {k: v/total for k, v in f1s.items()}
print(f"Pesos: {weights}")

# Ensemble
ensemble_probs_val = (
    weights['bert'] * bert_probs_val +
    weights['svc'] * svc_probs +
    weights['sgd'] * sgd_probs +
    weights['stacking'] * stacking_probs
)

baseline_preds = np.argmax(ensemble_probs_val, axis=1)
tuned_preds = apply_thresholds(ensemble_probs_val, THRESHOLDS)

print(f"Ensemble Baseline F1: {f1_score(val_labels, baseline_preds, average='macro'):.5f}")
print(f"Ensemble Threshold F1: {f1_score(val_labels, tuned_preds, average='macro'):.5f}")

In [ ]:
# ===== SUBMISSION =====
print("\n" + "="*60)
print("4. Submission")
print("="*60)

# Probabilidades de teste
bert_probs_test = []
with torch.no_grad():
    for batch in test_loader:
        outputs = model(batch['input_ids'].to(device), attention_mask=batch['attention_mask'].to(device))
        bert_probs_test.append(F.softmax(outputs.logits, dim=-1).cpu().numpy())
bert_probs_test = np.vstack(bert_probs_test)

svc_probs_test = svc_cal.predict_proba(X_test)
sgd_probs_test = sgd_cal.predict_proba(X_test)
stacking_probs_test = stacking.predict_proba(X_test)

ensemble_probs_test = (
    weights['bert'] * bert_probs_test +
    weights['svc'] * svc_probs_test +
    weights['sgd'] * sgd_probs_test +
    weights['stacking'] * stacking_probs_test
)

predictions = apply_thresholds(ensemble_probs_test, THRESHOLDS)

submission = pd.DataFrame({'ID': test_df['ID'], 'target': predictions})
submission.to_csv('submission.csv', index=False)
print(f"Submission salva!")
print(submission['target'].value_counts().sort_index())